In [38]:
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Dense, InputLayer, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1,L2,l1_l2
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow_addons as tfa
from keras import backend as K
from keras.metrics import MeanIoU, Accuracy

## Losses

In [39]:
def weighted_crosse(y_true, y_pred):
    smooth=1
    pixels = 512 * 512
    
    
    one_w = 1.0 -(K.sum(K.batch_flatten(y_true), axis=1)/pixels)
    zero_w = 1.0 -((pixels-K.sum(K.batch_flatten(y_true), axis=1))/pixels)

    initial_loss = tf.losses.binary_crossentropy(y_true, y_pred)
    
    zero_loss = K.mean( K.batch_flatten(tf.squeeze(tf.cast(y_true == 0, tf.float32), axis = -1) * initial_loss), axis=1)
    one_loss = K.mean( K.batch_flatten(tf.squeeze(tf.cast(y_true == 1, tf.float32), axis = -1) * initial_loss), axis=1)
    
    zero_weighted_loss = (one_w * 100) * zero_loss
    one_weighted_loss = (zero_w * 100) * one_loss
    
    return K.mean(zero_weighted_loss) + K.mean(one_weighted_loss)

In [40]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

In [41]:
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1+K.epsilon())) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed

In [42]:
def focal_dice_crosse(y_true, y_pred):
    bce = BinaryCrossentropy()
    return bce(y_true, y_pred) + dice_coef_loss(y_true, y_pred) + focal_loss()(y_true, y_pred)

In [43]:
def mean_iou(y_true, y_pred):
    y_pred = K.cast(K.greater(y_pred, .5), dtype='float32') # .5 is the threshold
    ones = K.sum(K.flatten(y_pred * y_true)) / K.sum(K.flatten(y_true))
    y_pred = K.cast(K.equal(y_pred, 0), dtype='float32')
    y_true = K.cast(K.equal(y_true, 0), dtype='float32')
    zeros = K.sum(K.flatten(y_pred * y_true)) / K.sum(K.flatten(y_true))
    return (ones + zeros)/2.0

## Models

In [44]:
#model1
filters = [32,32,64,64,128,128]
k_size = [5,5,4,3,3,2]

filters_t = [128,64,32,32,16,1]
k_size_t = [2,3,3,4,5,5]
activation = ['relu','relu','relu','relu','relu','sigmoid']

model1 = models.Sequential()

#encoder
model1.add(InputLayer(input_shape = (512,512,1)))

for f,k in zip(filters,k_size): 
    model1.add(Conv2D(f, kernel_size = k, strides=(1,1), padding="same", activation='relu'))
    #autoenc.add(BatchNormalization())
    model1.add(MaxPooling2D(pool_size=(2,2)))

#decoder
for f,k,a in zip(filters_t,k_size_t,activation): 
    model1.add(Conv2DTranspose(f, kernel_size=k, strides=(2,2), padding="same", activation=a))
    #autoenc.add(BatchNormalization())

model1.compile(optimizer = Adam(learning_rate=1e-3), loss=focal_dice_crosse, metrics = [mean_iou, 'accuracy'], run_eagerly=True)